In [1]:
import numpy as np
import jax.numpy as jnp
import mbirjax
import os
import matplotlib.pyplot as plt

In [1]:
import numpy as np
import jax.numpy as jnp
import mbirjax
import os
import matplotlib.pyplot as plt

# Define dataset directory
output_dir = "xray_dataset"
os.makedirs(output_dir, exist_ok=True)

# Choose the geometry type
geometry_type = 'parallel'  # 'cone' or 'parallel'

# Set parameters for the problem size
num_views = 64
num_det_rows = 40
num_det_channels = 128

# Set angles for projections
start_angle = -np.pi / 2
end_angle = np.pi / 2
angles = jnp.linspace(start_angle, end_angle, num_views, endpoint=False)

# Initialize the CT model
if geometry_type == 'parallel':
    ct_model = mbirjax.ParallelBeamModel((num_views, num_det_rows, num_det_channels), angles)
elif geometry_type == 'cone':
    source_detector_dist = 4 * num_det_channels
    source_iso_dist = source_detector_dist
    ct_model = mbirjax.ConeBeamModel((num_views, num_det_rows, num_det_channels), angles, source_detector_dist, source_iso_dist)
else:
    raise ValueError('Invalid geometry type')

# Generate 3D Shepp-Logan phantom
print("Generating Shepp-Logan phantom...")
phantom = ct_model.gen_modified_3d_sl_phantom()

# Generate synthetic sinogram (X-ray projections)
print("Generating sinogram...")
sinogram = ct_model.forward_project(phantom)
sinogram = np.array(sinogram)


# Save sinogram as PNG images
png_dir = os.path.join(output_dir, "png_projections")
os.makedirs(png_dir, exist_ok=True)

for i in range(num_views):
    plt.imshow(sinogram[i], cmap='gray', aspect='auto')
    plt.axis('off')
    plt.savefig(os.path.join(png_dir, f"projection_{i:03d}.png"), bbox_inches='tight', pad_inches=0)
    plt.close()

print(f"X-ray projections saved as PNG images in {png_dir}")

# Load the dataset for verification
data = np.load(os.path.join(output_dir, "xray_projections.npz"))
loaded_sinogram = data['sinogram']
loaded_angles = data['angles']
print("Dataset loaded successfully.")
print("Sinogram shape:", loaded_sinogram.shape)
print("Angles shape:", loaded_angles.shape)


Estimated memory required = 0.183GB full, 0.012GB update
Using TFRT_CPU_0 for main memory, TFRT_CPU_0 as worker.
views_per_batch = 256; pixels_per_batch = 2048
Generating Shepp-Logan phantom...
Generating sinogram...
X-ray projections saved as PNG images in xray_dataset/png_projections
Dataset loaded successfully.
Sinogram shape: (64, 40, 128)
Angles shape: (64,)
